In [447]:
from sympy import *
from sympy.vector import CoordSys3D, Del
from sympy.simplify.epathtools import EPath

from sympy.simplify.fu import *

In [448]:
R = CoordSys3D('R')
delop = Del()
x = R.x
y = R.y

eps, theta, r = symbols('epsilon theta r')
nx = Function('n_x')(x, y)
ny = Function('n_y')(x, y)

In [449]:
n = nx * R.i + ny * R.j
n

(n_x(R.x, R.y))*R.i + (n_y(R.x, R.y))*R.j

In [512]:
A, B = symbols('A B')

### Calculate charge density

In [450]:
splay = simplify( delop.dot(n).doit()**2 )
splay

(Derivative(n_x(R.x, R.y), R.x) + Derivative(n_y(R.x, R.y), R.y))**2

In [451]:
bend = simplify( n.cross( delop.cross(n).doit() ).magnitude()**2 )
bend = bend.subs(nx**2 + ny**2, 1)
bend

(Derivative(n_x(R.x, R.y), R.y) - Derivative(n_y(R.x, R.y), R.x))**2

In [452]:
g = (1 + eps) * splay + (1 - eps) * bend
g

(1 - epsilon)*(Derivative(n_x(R.x, R.y), R.y) - Derivative(n_y(R.x, R.y), R.x))**2 + (epsilon + 1)*(Derivative(n_x(R.x, R.y), R.x) + Derivative(n_y(R.x, R.y), R.y))**2

### Calculate in terms of director angle

In [453]:
phi = Function('phi')(x, y)
phi

phi(R.x, R.y)

In [454]:
g = simplify( g.subs({nx: cos(phi), ny: sin(phi)}) )
g

(1 - epsilon)*(sin(phi(R.x, R.y))*Derivative(phi(R.x, R.y), R.y) + cos(phi(R.x, R.y))*Derivative(phi(R.x, R.y), R.x))**2 + (epsilon + 1)*(-sin(phi(R.x, R.y))*Derivative(phi(R.x, R.y), R.x) + cos(phi(R.x, R.y))*Derivative(phi(R.x, R.y), R.y))**2

### Change to [polar coordinates](https://www.wikiwand.com/en/Polar_coordinate_system#Differential_calculus)

In [455]:
phi_p = Function('phi')(theta)

In [456]:
dphi_dx = cos(theta) * phi_p.diff(r) - (1/r) * sin(theta) * phi_p.diff(theta)
dphi_dy = sin(theta) * phi_p.diff(r) + (1/r) * cos(theta) * phi_p.diff(theta)

In [457]:
g_polar = g.subs({phi.diff(x) : dphi_dx, phi.diff(y) : dphi_dy, phi : phi_p})
g_polar = simplify(g_polar)
display(g_polar)

((1 - epsilon)*sin(theta - phi(theta))**2 + (epsilon + 1)*cos(theta - phi(theta))**2)*Derivative(phi(theta), theta)**2/r**2

### Plug it into [Euler-Lagrange equation](https://www.wikiwand.com/en/Euler%E2%80%93Lagrange_equation#Single_function_of_several_variables_with_single_derivative)

In [458]:
euler_lagrange = simplify(g_polar.diff(phi_p) - g_polar.diff(phi_p.diff(theta)).diff(theta))
euler_lagrange = trigsimp(factor(euler_lagrange, eps))
euler_lagrange = simplify(euler_lagrange * -Rational(1, 2) * r**2)
euler_lagrange = factor(euler_lagrange, eps, phi_p.diff(theta, 2), sin(2*theta - 2*phi_p))
display(euler_lagrange)

epsilon*(Derivative(phi(theta), theta)**2 - 2*Derivative(phi(theta), theta))*sin(2*theta - 2*phi(theta)) + epsilon*cos(2*theta - 2*phi(theta))*Derivative(phi(theta), (theta, 2)) + Derivative(phi(theta), (theta, 2))

### Assume ansatz

In [459]:
from sympy.simplify.fu import *

In [460]:
q, C = symbols('q C')
ansatz = q * theta + eps * C * sin(2 * (1 - q) * theta)

In [461]:
ansatz_euler_lagrange = simplify(euler_lagrange.subs(phi_p, ansatz))
ansatz_euler_lagrange = expand(ansatz_euler_lagrange).subs({eps**2: 0, eps**3: 0})
display(ansatz_euler_lagrange)

4*C*epsilon*q**2*sin(2*q*theta - 2*theta) - 8*C*epsilon*q*sin(2*q*theta - 2*theta) + 4*C*epsilon*sin(2*q*theta - 2*theta) + epsilon*q**2*sin(2*C*epsilon*sin(2*q*theta - 2*theta) - 2*q*theta + 2*theta) - 2*epsilon*q*sin(2*C*epsilon*sin(2*q*theta - 2*theta) - 2*q*theta + 2*theta)

In [462]:
sin_subs = {sin(2*C*eps*sin(2*q*theta - 2*theta)
                - 2*q*theta + 2*theta):
                sin(2*C*eps*sin(2*q*theta - 2*theta))
                * cos(2*theta - 2*q*theta)
                +
                sin(2*theta - 2*q*theta)
                * cos(2*C*eps*sin(2*q*theta - 2*theta))}

ansatz_euler_lagrange = ansatz_euler_lagrange.subs(sin_subs)
display(ansatz_euler_lagrange)

4*C*epsilon*q**2*sin(2*q*theta - 2*theta) - 8*C*epsilon*q*sin(2*q*theta - 2*theta) + 4*C*epsilon*sin(2*q*theta - 2*theta) + epsilon*q**2*(sin(2*C*epsilon*sin(2*q*theta - 2*theta))*cos(2*q*theta - 2*theta) - sin(2*q*theta - 2*theta)*cos(2*C*epsilon*sin(2*q*theta - 2*theta))) - 2*epsilon*q*(sin(2*C*epsilon*sin(2*q*theta - 2*theta))*cos(2*q*theta - 2*theta) - sin(2*q*theta - 2*theta)*cos(2*C*epsilon*sin(2*q*theta - 2*theta)))

In [463]:
trig_subs = {sin(2*C*eps*sin(2*q*theta - 2*theta)):
             2*C*eps*sin(2*q*theta - 2*theta),
             cos(2*C*eps*sin(2*q*theta - 2*theta)):
             1}
                 
ansatz_euler_lagrange = expand(ansatz_euler_lagrange.subs(trig_subs))
ansatz_euler_lagrange = ansatz_euler_lagrange.subs(eps**2, 0)
display(ansatz_euler_lagrange)

4*C*epsilon*q**2*sin(2*q*theta - 2*theta) - 8*C*epsilon*q*sin(2*q*theta - 2*theta) + 4*C*epsilon*sin(2*q*theta - 2*theta) - epsilon*q**2*sin(2*q*theta - 2*theta) + 2*epsilon*q*sin(2*q*theta - 2*theta)

In [464]:
sol = solve(ansatz_euler_lagrange, C)
display(sol[0])

q*(q - 2)/(4*(q**2 - 2*q + 1))

In [465]:
single_defect_approximation = ansatz.subs(C, sol[0])
display(single_defect_approximation)

epsilon*q*(q - 2)*sin(theta*(2 - 2*q))/(4*(q**2 - 2*q + 1)) + q*theta

### Above is the first-order approximation to a single defect under the Dzyaloshinskii solution

## Two-defect approximation

### First, Euler Lagrange in cartesian coordinates

In [466]:
euler_lagrange = -simplify(g.diff(phi) 
                           - (g.diff(phi.diff(x)).diff(x) 
                              + g.diff(phi.diff(y)).diff(y))) / 2
euler_lagrange = factor(euler_lagrange, phi.diff(x, 2), phi.diff(y, 2))
euler_lagrange = trigsimp(euler_lagrange)
euler_lagrange = simplify(euler_lagrange)
euler_lagrange = simplify(TR7(TR5(euler_lagrange)))
euler_lagrange = factor(euler_lagrange, sin(2*phi), cos(2*phi), eps)
display(euler_lagrange)

epsilon*(-2*Derivative(phi(R.x, R.y), R.x)*Derivative(phi(R.x, R.y), R.y) - Derivative(phi(R.x, R.y), (R.x, 2)) + Derivative(phi(R.x, R.y), (R.y, 2)))*cos(2*phi(R.x, R.y)) + epsilon*(Derivative(phi(R.x, R.y), R.x)**2 - Derivative(phi(R.x, R.y), R.y)**2 - 2*Derivative(phi(R.x, R.y), R.x, R.y))*sin(2*phi(R.x, R.y)) + Derivative(phi(R.x, R.y), (R.x, 2)) + Derivative(phi(R.x, R.y), (R.y, 2))

### Now we need to substitute the Ansatz

In [467]:
phi_c = Function('phi_c')(x, y)
theta_1 = Function('theta_1')(x, y)
theta_2 = Function('theta_2')(x, y)
r_1 = Function('r_1')(x, y)
r_2 = Function('r_2')(x, y)

q1, q2 = symbols('q_1 q_2')

ansatz = q1 * theta_1 + q2 * theta_2 + eps * phi_c
display(ansatz)

epsilon*phi_c(R.x, R.y) + q_1*theta_1(R.x, R.y) + q_2*theta_2(R.x, R.y)

In [468]:
euler_lagrange_p = euler_lagrange.subs(phi, ansatz)
euler_lagrange_p = simplify(euler_lagrange_p)
display(euler_lagrange_p)

-epsilon*(2*epsilon*Derivative(phi_c(R.x, R.y), R.x, R.y) + 2*q_1*Derivative(theta_1(R.x, R.y), R.x, R.y) + 2*q_2*Derivative(theta_2(R.x, R.y), R.x, R.y) - (epsilon*Derivative(phi_c(R.x, R.y), R.x) + q_1*Derivative(theta_1(R.x, R.y), R.x) + q_2*Derivative(theta_2(R.x, R.y), R.x))**2 + (epsilon*Derivative(phi_c(R.x, R.y), R.y) + q_1*Derivative(theta_1(R.x, R.y), R.y) + q_2*Derivative(theta_2(R.x, R.y), R.y))**2)*sin(2*epsilon*phi_c(R.x, R.y) + 2*q_1*theta_1(R.x, R.y) + 2*q_2*theta_2(R.x, R.y)) - epsilon*(epsilon*Derivative(phi_c(R.x, R.y), (R.x, 2)) - epsilon*Derivative(phi_c(R.x, R.y), (R.y, 2)) + q_1*Derivative(theta_1(R.x, R.y), (R.x, 2)) - q_1*Derivative(theta_1(R.x, R.y), (R.y, 2)) + q_2*Derivative(theta_2(R.x, R.y), (R.x, 2)) - q_2*Derivative(theta_2(R.x, R.y), (R.y, 2)) + 2*(epsilon*Derivative(phi_c(R.x, R.y), R.x) + q_1*Derivative(theta_1(R.x, R.y), R.x) + q_2*Derivative(theta_2(R.x, R.y), R.x))*(epsilon*Derivative(phi_c(R.x, R.y), R.y) + q_1*Derivative(theta_1(R.x, R.y), R.y) +

In [469]:
euler_lagrange_p = euler_lagrange_p.subs({theta_1.diff(x): -1/r_1 * sin(theta_1),
                                          theta_2.diff(x): -1/r_2 * sin(theta_2),
                                          theta_1.diff(y): 1/r_1 * cos(theta_1),
                                          theta_2.diff(y): 1/r_2 * cos(theta_2)})

In [470]:
display(euler_lagrange_p)

-epsilon*(2*epsilon*Derivative(phi_c(R.x, R.y), R.x, R.y) + 2*q_1*Derivative(-sin(theta_1(R.x, R.y))/r_1(R.x, R.y), R.y) + 2*q_2*Derivative(-sin(theta_2(R.x, R.y))/r_2(R.x, R.y), R.y) - (epsilon*Derivative(phi_c(R.x, R.y), R.x) - q_1*sin(theta_1(R.x, R.y))/r_1(R.x, R.y) - q_2*sin(theta_2(R.x, R.y))/r_2(R.x, R.y))**2 + (epsilon*Derivative(phi_c(R.x, R.y), R.y) + q_1*cos(theta_1(R.x, R.y))/r_1(R.x, R.y) + q_2*cos(theta_2(R.x, R.y))/r_2(R.x, R.y))**2)*sin(2*epsilon*phi_c(R.x, R.y) + 2*q_1*theta_1(R.x, R.y) + 2*q_2*theta_2(R.x, R.y)) - epsilon*(epsilon*Derivative(phi_c(R.x, R.y), (R.x, 2)) - epsilon*Derivative(phi_c(R.x, R.y), (R.y, 2)) + q_1*Derivative(-sin(theta_1(R.x, R.y))/r_1(R.x, R.y), R.x) - q_1*Derivative(cos(theta_1(R.x, R.y))/r_1(R.x, R.y), R.y) + q_2*Derivative(-sin(theta_2(R.x, R.y))/r_2(R.x, R.y), R.x) - q_2*Derivative(cos(theta_2(R.x, R.y))/r_2(R.x, R.y), R.y) + 2*(epsilon*Derivative(phi_c(R.x, R.y), R.x) - q_1*sin(theta_1(R.x, R.y))/r_1(R.x, R.y) - q_2*sin(theta_2(R.x, R.y))

In [471]:
euler_lagrange_p = euler_lagrange_p.doit()
euler_lagrange_p = euler_lagrange_p.subs({theta_1.diff(x): -1/r_1 * sin(theta_1),
                                          theta_2.diff(x): -1/r_2 * sin(theta_2),
                                          theta_1.diff(y): 1/r_1 * cos(theta_1),
                                          theta_2.diff(y): 1/r_2 * cos(theta_2)})
for arg in euler_lagrange_p.args:
    display(arg)

epsilon*Derivative(phi_c(R.x, R.y), (R.x, 2))

epsilon*Derivative(phi_c(R.x, R.y), (R.y, 2))

q_1*(sin(theta_1(R.x, R.y))*cos(theta_1(R.x, R.y))/r_1(R.x, R.y)**2 + sin(theta_1(R.x, R.y))*Derivative(r_1(R.x, R.y), R.x)/r_1(R.x, R.y)**2)

q_1*(-sin(theta_1(R.x, R.y))*cos(theta_1(R.x, R.y))/r_1(R.x, R.y)**2 - cos(theta_1(R.x, R.y))*Derivative(r_1(R.x, R.y), R.y)/r_1(R.x, R.y)**2)

q_2*(sin(theta_2(R.x, R.y))*cos(theta_2(R.x, R.y))/r_2(R.x, R.y)**2 + sin(theta_2(R.x, R.y))*Derivative(r_2(R.x, R.y), R.x)/r_2(R.x, R.y)**2)

q_2*(-sin(theta_2(R.x, R.y))*cos(theta_2(R.x, R.y))/r_2(R.x, R.y)**2 - cos(theta_2(R.x, R.y))*Derivative(r_2(R.x, R.y), R.y)/r_2(R.x, R.y)**2)

-epsilon*(2*epsilon*Derivative(phi_c(R.x, R.y), R.x, R.y) + 2*q_1*(sin(theta_1(R.x, R.y))*Derivative(r_1(R.x, R.y), R.y)/r_1(R.x, R.y)**2 - cos(theta_1(R.x, R.y))**2/r_1(R.x, R.y)**2) + 2*q_2*(sin(theta_2(R.x, R.y))*Derivative(r_2(R.x, R.y), R.y)/r_2(R.x, R.y)**2 - cos(theta_2(R.x, R.y))**2/r_2(R.x, R.y)**2) - (epsilon*Derivative(phi_c(R.x, R.y), R.x) - q_1*sin(theta_1(R.x, R.y))/r_1(R.x, R.y) - q_2*sin(theta_2(R.x, R.y))/r_2(R.x, R.y))**2 + (epsilon*Derivative(phi_c(R.x, R.y), R.y) + q_1*cos(theta_1(R.x, R.y))/r_1(R.x, R.y) + q_2*cos(theta_2(R.x, R.y))/r_2(R.x, R.y))**2)*sin(2*epsilon*phi_c(R.x, R.y) + 2*q_1*theta_1(R.x, R.y) + 2*q_2*theta_2(R.x, R.y))

-epsilon*(epsilon*Derivative(phi_c(R.x, R.y), (R.x, 2)) - epsilon*Derivative(phi_c(R.x, R.y), (R.y, 2)) - q_1*(-sin(theta_1(R.x, R.y))*cos(theta_1(R.x, R.y))/r_1(R.x, R.y)**2 - cos(theta_1(R.x, R.y))*Derivative(r_1(R.x, R.y), R.y)/r_1(R.x, R.y)**2) + q_1*(sin(theta_1(R.x, R.y))*cos(theta_1(R.x, R.y))/r_1(R.x, R.y)**2 + sin(theta_1(R.x, R.y))*Derivative(r_1(R.x, R.y), R.x)/r_1(R.x, R.y)**2) - q_2*(-sin(theta_2(R.x, R.y))*cos(theta_2(R.x, R.y))/r_2(R.x, R.y)**2 - cos(theta_2(R.x, R.y))*Derivative(r_2(R.x, R.y), R.y)/r_2(R.x, R.y)**2) + q_2*(sin(theta_2(R.x, R.y))*cos(theta_2(R.x, R.y))/r_2(R.x, R.y)**2 + sin(theta_2(R.x, R.y))*Derivative(r_2(R.x, R.y), R.x)/r_2(R.x, R.y)**2) + 2*(epsilon*Derivative(phi_c(R.x, R.y), R.x) - q_1*sin(theta_1(R.x, R.y))/r_1(R.x, R.y) - q_2*sin(theta_2(R.x, R.y))/r_2(R.x, R.y))*(epsilon*Derivative(phi_c(R.x, R.y), R.y) + q_1*cos(theta_1(R.x, R.y))/r_1(R.x, R.y) + q_2*cos(theta_2(R.x, R.y))/r_2(R.x, R.y)))*cos(2*epsilon*phi_c(R.x, R.y) + 2*q_1*theta_1(R.x, R.y)

In [472]:
euler_lagrange_p = euler_lagrange_p.subs({r_1.diff(x): cos(theta_1),
                                          r_2.diff(x): cos(theta_2),
                                          r_1.diff(y): sin(theta_1),
                                          r_2.diff(y): sin(theta_2)})

for arg in euler_lagrange_p.args:
    display(arg)

epsilon*Derivative(phi_c(R.x, R.y), (R.x, 2))

epsilon*Derivative(phi_c(R.x, R.y), (R.y, 2))

-epsilon*(2*epsilon*Derivative(phi_c(R.x, R.y), R.x, R.y) + 2*q_1*(sin(theta_1(R.x, R.y))**2/r_1(R.x, R.y)**2 - cos(theta_1(R.x, R.y))**2/r_1(R.x, R.y)**2) + 2*q_2*(sin(theta_2(R.x, R.y))**2/r_2(R.x, R.y)**2 - cos(theta_2(R.x, R.y))**2/r_2(R.x, R.y)**2) - (epsilon*Derivative(phi_c(R.x, R.y), R.x) - q_1*sin(theta_1(R.x, R.y))/r_1(R.x, R.y) - q_2*sin(theta_2(R.x, R.y))/r_2(R.x, R.y))**2 + (epsilon*Derivative(phi_c(R.x, R.y), R.y) + q_1*cos(theta_1(R.x, R.y))/r_1(R.x, R.y) + q_2*cos(theta_2(R.x, R.y))/r_2(R.x, R.y))**2)*sin(2*epsilon*phi_c(R.x, R.y) + 2*q_1*theta_1(R.x, R.y) + 2*q_2*theta_2(R.x, R.y))

-epsilon*(epsilon*Derivative(phi_c(R.x, R.y), (R.x, 2)) - epsilon*Derivative(phi_c(R.x, R.y), (R.y, 2)) + 4*q_1*sin(theta_1(R.x, R.y))*cos(theta_1(R.x, R.y))/r_1(R.x, R.y)**2 + 4*q_2*sin(theta_2(R.x, R.y))*cos(theta_2(R.x, R.y))/r_2(R.x, R.y)**2 + 2*(epsilon*Derivative(phi_c(R.x, R.y), R.x) - q_1*sin(theta_1(R.x, R.y))/r_1(R.x, R.y) - q_2*sin(theta_2(R.x, R.y))/r_2(R.x, R.y))*(epsilon*Derivative(phi_c(R.x, R.y), R.y) + q_1*cos(theta_1(R.x, R.y))/r_1(R.x, R.y) + q_2*cos(theta_2(R.x, R.y))/r_2(R.x, R.y)))*cos(2*epsilon*phi_c(R.x, R.y) + 2*q_1*theta_1(R.x, R.y) + 2*q_2*theta_2(R.x, R.y))

In [473]:
euler_lagrange_p = expand(euler_lagrange_p)
euler_lagrange_p = euler_lagrange_p.subs({eps**2 : 0, eps**3 : 0})
euler_lagrange_p = simplify(euler_lagrange_p)

display(euler_lagrange_p)

epsilon*(-q_1**2*r_2(R.x, R.y)**2*sin(2*epsilon*phi_c(R.x, R.y) + 2*q_1*theta_1(R.x, R.y) + 2*q_2*theta_2(R.x, R.y) - 2*theta_1(R.x, R.y)) - 2*q_1*q_2*r_1(R.x, R.y)*r_2(R.x, R.y)*sin(2*epsilon*phi_c(R.x, R.y) + 2*q_1*theta_1(R.x, R.y) + 2*q_2*theta_2(R.x, R.y) - theta_1(R.x, R.y) - theta_2(R.x, R.y)) + 2*q_1*r_2(R.x, R.y)**2*sin(2*epsilon*phi_c(R.x, R.y) + 2*q_1*theta_1(R.x, R.y) + 2*q_2*theta_2(R.x, R.y) - 2*theta_1(R.x, R.y)) - q_2**2*r_1(R.x, R.y)**2*sin(2*epsilon*phi_c(R.x, R.y) + 2*q_1*theta_1(R.x, R.y) + 2*q_2*theta_2(R.x, R.y) - 2*theta_2(R.x, R.y)) + 2*q_2*r_1(R.x, R.y)**2*sin(2*epsilon*phi_c(R.x, R.y) + 2*q_1*theta_1(R.x, R.y) + 2*q_2*theta_2(R.x, R.y) - 2*theta_2(R.x, R.y)) + r_1(R.x, R.y)**2*r_2(R.x, R.y)**2*Derivative(phi_c(R.x, R.y), (R.x, 2)) + r_1(R.x, R.y)**2*r_2(R.x, R.y)**2*Derivative(phi_c(R.x, R.y), (R.y, 2)))/(r_1(R.x, R.y)**2*r_2(R.x, R.y)**2)

## Great, now that we've got the governing equation let's solve

In [474]:
q1, q2, eps, r1, r2, theta1, theta2 = symbols('q_1 q_2 epsilon r_1 r_2 theta_1 theta_2')

In [475]:
eq_rhs = ( (q1*(2 - q1)*eps / r1**2) * sin(2*(1 - q1)*theta1 - 2*q2*theta2)
           + (q2*(2 - q2)*eps / r2**2) * sin(2*(1 - q2)*theta2 - 2*q1*theta1)
           - (2*eps*q1*q2 / (r1*r2)) * sin((1 - 2*q1)*theta1 + (1 - 2*q2)*theta2) )
display(eq_rhs)

-2*epsilon*q_1*q_2*sin(theta_1*(1 - 2*q_1) + theta_2*(1 - 2*q_2))/(r_1*r_2) - epsilon*q_1*(2 - q_1)*sin(2*q_2*theta_2 - theta_1*(2 - 2*q_1))/r_1**2 - epsilon*q_2*(2 - q_2)*sin(2*q_1*theta_1 - theta_2*(2 - 2*q_2))/r_2**2

In [476]:
eq_rhs = TR10(eq_rhs)
display(eq_rhs)

-2*epsilon*q_1*q_2*(sin(theta_1*(1 - 2*q_1))*cos(theta_2*(1 - 2*q_2)) + sin(theta_2*(1 - 2*q_2))*cos(theta_1*(1 - 2*q_1)))/(r_1*r_2) - epsilon*q_1*(2 - q_1)*(sin(2*q_2*theta_2)*cos(theta_1*(2 - 2*q_1)) - sin(theta_1*(2 - 2*q_1))*cos(2*q_2*theta_2))/r_1**2 - epsilon*q_2*(2 - q_2)*(sin(2*q_1*theta_1)*cos(theta_2*(2 - 2*q_2)) - sin(theta_2*(2 - 2*q_2))*cos(2*q_1*theta_1))/r_2**2

### Far from both defects approximations

In [477]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

In [478]:
plt.style.use('science')
mpl.rcParams['figure.dpi'] = 200

In [479]:
# r = np.array([3, 5])
# d = np.array([2, 0])
# r1 = r + d/2
# r2 = r - d/2

# delta = np.array([0.2, 0.2])

# plt.arrow(-d[0]/2, -d[1]/2, r1[0], r1[1])
# plt.text(r1[0]/2 - d[0]/2 - delta[0], r1[1]/2 - d[1]/2 + delta[1], r'$r_1$')

# plt.arrow(0, 0, r[0], r[1])
# plt.text(r[0]/4 - delta[0], r[1]/4 + delta[1], r'$r$')

# plt.arrow(d[0]/2, d[1]/2, r2[0], r2[1])
# plt.text(r2[0]/2 + d[0]/2 + delta[0], r2[1] / 2 + d[1]/2 - delta[0], r'$r_2$')


# plt.xlim([-2, 5])
# plt.ylim([-1, 6])
# plt.show()

$\mathbf{r}_2 + \frac{\mathbf{d}}{2} = \mathbf{r} \implies \mathbf{r}_2 = \mathbf{r} - \frac{\mathbf{d}}{2}$

$\mathbf{r}_1 - \frac{\mathbf{d}}{2} = \mathbf{r} \implies \mathbf{r}_1 = \mathbf{r} + \frac{\mathbf{d}}{2}$

$(\mathbf{r} \pm \frac{\mathbf{d}}{2})^2 = r^2 + \frac{d^2}{4} \pm rd \cos\theta$

$r_2 = \sqrt{r^2 + \frac{d^2}{4} - rd\cos\theta}$

$r_1 = \sqrt{r^2 + \frac{d^2}{4} + rd\cos\theta}$

$\mathbf{r_2} \cdot \mathbf{d} = r_2 d \cos\theta_2 = rd \cos\theta - \frac{d^2}{2} \implies \cos\theta_2 = \frac{r\cos\theta - \frac{d}{2}}{\sqrt{r^2 + \frac{d^2}{4} - rd\cos\theta}}$

$\mathbf{r_1} \cdot \mathbf{d} = r_1 d \cos\theta_1 = rd\cos\theta + \frac{d^2}{2} \implies \cos\theta_1 = \frac{r\cos\theta + \frac{d}{2}}{\sqrt{r^2 + \frac{d^2}{4} + rd\cos\theta}}$

$|\mathbf{r_2}\times \mathbf{d}| = r_2 d \sin\theta_2 = rd\sin\theta \implies \sin\theta_2 = \frac{r\sin\theta}{\sqrt{r^2 + \frac{d^2}{4} - rd\cos\theta}}$

$|\mathbf{r_1}\times \mathbf{d}| = r_1 d \sin\theta_1 = rd\sin\theta \implies \sin\theta_1 = \frac{r\sin\theta}{\sqrt{r^2 + \frac{d^2}{4} + rd\cos\theta}}$

## Choose $q_1 = +1/2$ and $q_2 = -1/2$

In [480]:
eq_rhs_pos = eq_rhs.subs({q1: Rational(1, 2), q2: -Rational(1, 2)})
display(eq_rhs_pos)

5*epsilon*(sin(theta_1)*cos(3*theta_2) - sin(3*theta_2)*cos(theta_1))/(4*r_2**2) + epsilon*sin(2*theta_2)/(2*r_1*r_2) - 3*epsilon*(-sin(theta_1)*cos(theta_2) - sin(theta_2)*cos(theta_1))/(4*r_1**2)

In [481]:
eq_rhs_pos = TR11(eq_rhs_pos)
display(eq_rhs_pos)

5*epsilon*(sin(theta_1)*cos(3*theta_2) - sin(3*theta_2)*cos(theta_1))/(4*r_2**2) + epsilon*sin(theta_2)*cos(theta_2)/(r_1*r_2) - 3*epsilon*(-sin(theta_1)*cos(theta_2) - sin(theta_2)*cos(theta_1))/(4*r_1**2)

In [482]:
eq_rhs_pos = eq_rhs_pos.subs({sin(3*theta2): 3*sin(theta2) - 4*sin(theta2)**3,
                              cos(3*theta2): 4*cos(theta2)**3 - 3*cos(theta2)})
eq_rhs_pos = expand(eq_rhs_pos)
display(eq_rhs_pos)

5*epsilon*sin(theta_1)*cos(theta_2)**3/r_2**2 - 15*epsilon*sin(theta_1)*cos(theta_2)/(4*r_2**2) + 5*epsilon*sin(theta_2)**3*cos(theta_1)/r_2**2 - 15*epsilon*sin(theta_2)*cos(theta_1)/(4*r_2**2) + epsilon*sin(theta_2)*cos(theta_2)/(r_1*r_2) + 3*epsilon*sin(theta_1)*cos(theta_2)/(4*r_1**2) + 3*epsilon*sin(theta_2)*cos(theta_1)/(4*r_1**2)

In [483]:
theta = symbols('theta')
r, d = symbols('r d', positive=True)

In [484]:
subs_dict = {sin(theta1): (r * sin(theta) / r1),
             sin(theta2): r * sin(theta) / r2,
             cos(theta1): (r * cos(theta) + d/2) / r1,
             cos(theta2): (r * cos(theta) - d/2) / r2}
eq_rhs_pos = eq_rhs_pos.subs(subs_dict)
display(eq_rhs_pos)

5*epsilon*r**3*(d/2 + r*cos(theta))*sin(theta)**3/(r_1*r_2**5) - 11*epsilon*r*(-d/2 + r*cos(theta))*sin(theta)/(4*r_1*r_2**3) - 15*epsilon*r*(d/2 + r*cos(theta))*sin(theta)/(4*r_1*r_2**3) + 5*epsilon*r*(-d/2 + r*cos(theta))**3*sin(theta)/(r_1*r_2**5) + 3*epsilon*r*(-d/2 + r*cos(theta))*sin(theta)/(4*r_1**3*r_2) + 3*epsilon*r*(d/2 + r*cos(theta))*sin(theta)/(4*r_1**3*r_2)

In [485]:
subs_dict = {r1: sqrt(r**2 + d**2 / 4 + r*d*cos(theta)),
             r2: sqrt(r**2 + d**2 / 4 - r*d*cos(theta))}
eq_rhs_pos = eq_rhs_pos.subs(subs_dict)
eq_rhs_pos = simplify(eq_rhs_pos)
display(eq_rhs_pos)

16*d*epsilon*r*(-3*d**4*sin(theta) + 4*d**2*r**2*sin(3*theta) + 32*d*r**3*sin(2*theta) - 4*d*r**3*sin(4*theta) - 32*r**4*sin(3*theta))/((d**2 - 4*d*r*cos(theta) + 4*r**2)**(5/2)*(d**2 + 4*d*r*cos(theta) + 4*r**2)**(3/2))

## Assumption that we're making is that $r \gg d \implies d/r \ll 1$

## Take $x = d/r \implies d = rx$

In [486]:
x = symbols('x')
eq_rhs_pos = simplify(eq_rhs_pos.subs(d, x*r))
display(eq_rhs_pos)

-16*epsilon*x*(3*x**4*sin(theta) - 4*x**2*sin(3*theta) - 32*x*sin(2*theta) + 4*x*sin(4*theta) + 32*sin(3*theta))/(r**2*(x**2 - 4*x*cos(theta) + 4)**(5/2)*(x**2 + 4*x*cos(theta) + 4)**(3/2))

In [487]:
eq_rhs_series = eq_rhs_pos.series(x, 0, 3)
display(eq_rhs_series)

x**2*(2*epsilon*sin(2*theta)/r**2 - 2*epsilon*sin(3*theta)*cos(theta)/r**2 - epsilon*sin(4*theta)/(4*r**2)) - 2*epsilon*x*sin(3*theta)/r**2 + O(x**3)

In [488]:
simplify(TR8(eq_rhs_series.subs(x, d/r)))

(-8*d*epsilon*r**4*sin(3*theta) + d**2*epsilon*r**3*(4*sin(2*theta) - 5*sin(4*theta)) + O(d**3*r**4, d, r))/(4*r**7)

In [489]:
sol = A * sin(3*theta) / r

In [490]:
lap_sol = sol.diff(r, 2) + (1/r) * sol.diff(r) + (1/r**2) * sol.diff(theta, 2)
display(lap_sol)

-8*A*sin(3*theta)/r**3

### Hence, $A = d \epsilon / 4$

## Let's try $q_1 = -1/2, q_2 = 1/2$

In [491]:
eq_rhs_neg = eq_rhs.subs({q1: -Rational(1, 2), q2: Rational(1, 2)})
display(eq_rhs_neg)

-3*epsilon*(-sin(theta_1)*cos(theta_2) - sin(theta_2)*cos(theta_1))/(4*r_2**2) + epsilon*sin(2*theta_1)/(2*r_1*r_2) + 5*epsilon*(-sin(3*theta_1)*cos(theta_2) + sin(theta_2)*cos(3*theta_1))/(4*r_1**2)

In [492]:
eq_rhs_neg = TR11(eq_rhs_neg)
display(eq_rhs_neg)

-3*epsilon*(-sin(theta_1)*cos(theta_2) - sin(theta_2)*cos(theta_1))/(4*r_2**2) + epsilon*sin(theta_1)*cos(theta_1)/(r_1*r_2) + 5*epsilon*(-sin(3*theta_1)*cos(theta_2) + sin(theta_2)*cos(3*theta_1))/(4*r_1**2)

In [493]:
eq_rhs_neg = eq_rhs_neg.subs({sin(3*theta1): 3*sin(theta1) - 4*sin(theta1)**3,
                              cos(3*theta1): 4*cos(theta1)**3 - 3*cos(theta1)})
eq_rhs_neg = expand(eq_rhs_neg)
display(eq_rhs_neg)

3*epsilon*sin(theta_1)*cos(theta_2)/(4*r_2**2) + 3*epsilon*sin(theta_2)*cos(theta_1)/(4*r_2**2) + epsilon*sin(theta_1)*cos(theta_1)/(r_1*r_2) + 5*epsilon*sin(theta_1)**3*cos(theta_2)/r_1**2 - 15*epsilon*sin(theta_1)*cos(theta_2)/(4*r_1**2) + 5*epsilon*sin(theta_2)*cos(theta_1)**3/r_1**2 - 15*epsilon*sin(theta_2)*cos(theta_1)/(4*r_1**2)

In [494]:
subs_dict = {sin(theta1): (r * sin(theta) / r1),
             sin(theta2): r * sin(theta) / r2,
             cos(theta1): (r * cos(theta) + d/2) / r1,
             cos(theta2): (r * cos(theta) - d/2) / r2}
eq_rhs_neg = eq_rhs_neg.subs(subs_dict)
display(eq_rhs_neg)

5*epsilon*r**3*(-d/2 + r*cos(theta))*sin(theta)**3/(r_1**5*r_2) + 3*epsilon*r*(-d/2 + r*cos(theta))*sin(theta)/(4*r_1*r_2**3) + 3*epsilon*r*(d/2 + r*cos(theta))*sin(theta)/(4*r_1*r_2**3) - 15*epsilon*r*(-d/2 + r*cos(theta))*sin(theta)/(4*r_1**3*r_2) - 11*epsilon*r*(d/2 + r*cos(theta))*sin(theta)/(4*r_1**3*r_2) + 5*epsilon*r*(d/2 + r*cos(theta))**3*sin(theta)/(r_1**5*r_2)

In [495]:
subs_dict = {r1: sqrt(r**2 + d**2 / 4 + r*d*cos(theta)),
             r2: sqrt(r**2 + d**2 / 4 - r*d*cos(theta))}
eq_rhs_neg = eq_rhs_neg.subs(subs_dict)
eq_rhs_neg = simplify(eq_rhs_neg)
display(eq_rhs_neg)

16*d*epsilon*r*(3*d**4*sin(theta) - 4*d**2*r**2*sin(3*theta) + 32*d*r**3*sin(2*theta) - 4*d*r**3*sin(4*theta) + 32*r**4*sin(3*theta))/((d**2 - 4*d*r*cos(theta) + 4*r**2)**(3/2)*(d**2 + 4*d*r*cos(theta) + 4*r**2)**(5/2))

In [496]:
x = symbols('x')
eq_rhs_neg = simplify(eq_rhs_neg.subs(d, x*r))
display(eq_rhs_neg)

16*epsilon*x*(3*x**4*sin(theta) - 4*x**2*sin(3*theta) + 32*x*sin(2*theta) - 4*x*sin(4*theta) + 32*sin(3*theta))/(r**2*(x**2 - 4*x*cos(theta) + 4)**(3/2)*(x**2 + 4*x*cos(theta) + 4)**(5/2))

In [497]:
eq_rhs_series = eq_rhs_neg.series(x, 0, 2)
display(eq_rhs_series)

2*epsilon*x*sin(3*theta)/r**2 + O(x**2)

## Now for close to one defect, or $r_1/d \ll 1$

$\mathbf{r}_1 = \mathbf{r}_2 + \mathbf{d}$

$\mathbf{r}_2 = \mathbf{r}_1 - \mathbf{d}$

$\mathbf{r}_2 \cdot \mathbf{d} = r_2 d \cos \theta_2 = r_1 d \cos\theta_1 - d^2$

$\mathbf{r}_2 \times \mathbf{d} = r_2 d \sin\theta_2 = \mathbf{r}_1 \times \mathbf{d} = r_1 d \sin\theta_1$

$r_2 = \sqrt{r_1^2 + d^2 - r_1 d \cos\theta_1}$

### Solving for $q_1$

In [498]:
eq_rhs_close_pos = eq_rhs.subs({q1: Rational(1, 2), q2: -Rational(1, 2)})
eq_rhs_close_pos = TR11(eq_rhs_close_pos)
display(eq_rhs_close_pos)

5*epsilon*(sin(theta_1)*cos(3*theta_2) - sin(3*theta_2)*cos(theta_1))/(4*r_2**2) + epsilon*sin(theta_2)*cos(theta_2)/(r_1*r_2) - 3*epsilon*(-sin(theta_1)*cos(theta_2) - sin(theta_2)*cos(theta_1))/(4*r_1**2)

In [499]:
eq_rhs_close_pos = eq_rhs_close_pos.subs({sin(3*theta2): 3*sin(theta2) - 4*sin(theta2)**3,
                                          cos(3*theta2): 4*cos(theta2)**3 - 3*cos(theta2)})
display(eq_rhs_close_pos)

5*epsilon*(-(-4*sin(theta_2)**3 + 3*sin(theta_2))*cos(theta_1) + (4*cos(theta_2)**3 - 3*cos(theta_2))*sin(theta_1))/(4*r_2**2) + epsilon*sin(theta_2)*cos(theta_2)/(r_1*r_2) - 3*epsilon*(-sin(theta_1)*cos(theta_2) - sin(theta_2)*cos(theta_1))/(4*r_1**2)

In [500]:
subs_dict_1 = {cos(theta2): (r1 * cos(theta1) - d) / r2,
               sin(theta2): (r1 / r2) * sin(theta1)}
subs_dict_2 = {r2: sqrt(r1**2 + d**2 - r1*d*cos(theta1))}

eq_rhs_close_pos = eq_rhs_close_pos.subs(subs_dict_1)
eq_rhs_close_pos = eq_rhs_close_pos.subs(subs_dict_2)
eq_rhs_close_pos = simplify(eq_rhs_close_pos)
display(eq_rhs_close_pos)

d*epsilon*(-3*d**4 + 12*d**3*r_1*cos(theta_1) - 15*d**2*r_1**2*cos(theta_1)**2 - 15*d**2*r_1**2 + 6*d*r_1**3*cos(theta_1)**3 + 41*d*r_1**3*cos(theta_1) - 46*r_1**4*cos(theta_1)**2 + 8*r_1**4)*sin(theta_1)/(4*r_1**2*(d**2 - d*r_1*cos(theta_1) + r_1**2)**(5/2))

In [501]:
eq_rhs_close_pos = eq_rhs_close_pos.subs(r1, x * d)
display(eq_rhs_close_pos)

epsilon*(-46*d**4*x**4*cos(theta_1)**2 + 8*d**4*x**4 + 6*d**4*x**3*cos(theta_1)**3 + 41*d**4*x**3*cos(theta_1) - 15*d**4*x**2*cos(theta_1)**2 - 15*d**4*x**2 + 12*d**4*x*cos(theta_1) - 3*d**4)*sin(theta_1)/(4*d*x**2*(d**2*x**2 - d**2*x*cos(theta_1) + d**2)**(5/2))

In [502]:
eq_rhs_close_pos_series = eq_rhs_close_pos.series(x, 0, 0)
eq_rhs_close_pos_series = TR8(eq_rhs_close_pos_series).subs(x, r1/d)
display(eq_rhs_close_pos_series)

9*epsilon*sin(2*theta_1)/(16*d*r_1) - 3*epsilon*sin(theta_1)/(4*r_1**2) + O(1)

In [521]:
sol = A * r1 * sin(2*theta1) + B * sin(theta1)

lap_sol = sol.diff(r1, 2) + (1/r1) * sol.diff(r1) + (1/r1**2) * sol.diff(theta1, 2)
display(lap_sol)

A*sin(2*theta_1)/r_1 - (4*A*r_1*sin(2*theta_1) + B*sin(theta_1))/r_1**2

### Hence, $A = -\frac{3 \epsilon}{16 d}$, and $B = \frac{3\epsilon}{4}$

### Solving for $q_2$

In [503]:
eq_rhs_close_neg = eq_rhs.subs({q1: Rational(1, 2), q2: -Rational(1, 2)})
eq_rhs_close_neg = TR11(eq_rhs_close_neg)
display(eq_rhs_close_neg)

5*epsilon*(sin(theta_1)*cos(3*theta_2) - sin(3*theta_2)*cos(theta_1))/(4*r_2**2) + epsilon*sin(theta_2)*cos(theta_2)/(r_1*r_2) - 3*epsilon*(-sin(theta_1)*cos(theta_2) - sin(theta_2)*cos(theta_1))/(4*r_1**2)

In [504]:
eq_rhs_close_neg = eq_rhs_close_neg.subs({sin(3*theta2): 3*sin(theta2) - 4*sin(theta2)**3,
                                          cos(3*theta2): 4*cos(theta2)**3 - 3*cos(theta2)})
display(eq_rhs_close_neg)

5*epsilon*(-(-4*sin(theta_2)**3 + 3*sin(theta_2))*cos(theta_1) + (4*cos(theta_2)**3 - 3*cos(theta_2))*sin(theta_1))/(4*r_2**2) + epsilon*sin(theta_2)*cos(theta_2)/(r_1*r_2) - 3*epsilon*(-sin(theta_1)*cos(theta_2) - sin(theta_2)*cos(theta_1))/(4*r_1**2)

$\cos\theta_1 = \frac{r_2 \cos\theta_2 + d}{r_1}$

$\sin\theta_1 = \frac{r_2 \sin\theta_2}{r_1}$

$r_1 = \sqrt{r_2^2 + d^2 + r_2 d \cos\theta_2}$

In [505]:
subs_dict_1 = {cos(theta1): (r2 * cos(theta2) + d) / r1,
               sin(theta1): (r2 / r1) * sin(theta2)}
subs_dict_2 = {r1: sqrt(r2**2 + d**2 + r2*d*cos(theta2))}

eq_rhs_close_neg = eq_rhs_close_neg.subs(subs_dict_1)
eq_rhs_close_neg = eq_rhs_close_neg.subs(subs_dict_2)
eq_rhs_close_neg = simplify(eq_rhs_close_neg)
display(eq_rhs_close_neg)

epsilon*(3*r_2**2*(d + 2*r_2*cos(theta_2)) + 4*r_2*(d**2 + d*r_2*cos(theta_2) + r_2**2)*cos(theta_2) + 5*(r_2*cos(3*theta_2) + (d + r_2*cos(theta_2))*(4*sin(theta_2)**2 - 3))*(d**2 + d*r_2*cos(theta_2) + r_2**2))*sin(theta_2)/(4*r_2**2*(d**2 + d*r_2*cos(theta_2) + r_2**2)**(3/2))

In [506]:
eq_rhs_close_neg = eq_rhs_close_neg.subs(r2, x * d)
display(eq_rhs_close_neg)

epsilon*(3*d**2*x**2*(2*d*x*cos(theta_2) + d) + 4*d*x*(d**2*x**2 + d**2*x*cos(theta_2) + d**2)*cos(theta_2) + 5*(d*x*cos(3*theta_2) + (d*x*cos(theta_2) + d)*(4*sin(theta_2)**2 - 3))*(d**2*x**2 + d**2*x*cos(theta_2) + d**2))*sin(theta_2)/(4*d**2*x**2*(d**2*x**2 + d**2*x*cos(theta_2) + d**2)**(3/2))

In [507]:
eq_rhs_close_neg_series = eq_rhs_close_neg.series(x, 0, 0)
eq_rhs_close_neg_series = TR8(eq_rhs_close_neg_series).subs(x, r2/d)
display(eq_rhs_close_neg_series)

d*(5*epsilon*(-sin(2*theta_2) + sin(4*theta_2))/(8*d**2) + 5*epsilon*(2*sin(2*theta_2) - sin(4*theta_2))/(16*d**2) - 7*epsilon*sin(2*theta_2)/(16*d**2))/r_2 + d**2*(5*epsilon*(3*sin(theta_2)/4 - sin(3*theta_2)/4)/d**2 - 15*epsilon*sin(theta_2)/(4*d**2))/r_2**2 + O(1)

In [508]:
simplify(eq_rhs_close_neg_series)

(epsilon*r_2**2*(-7*sin(2*theta_2) + 5*sin(4*theta_2)) - 20*d*epsilon*r_2*sin(3*theta_2) + O(d*r_2**3, d, r_2))/(16*d*r_2**3)